In [3]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("../data/sealion/Train/train.csv")

In [9]:
df.head()

,train_id,adult_males,subadult_males,adult_females,juveniles,pups
0,0,62,12,486,42,344
1,1,2,20,0,12,0
2,2,2,0,38,20,0
3,3,8,5,41,7,38
4,4,6,9,2,0,0


In [10]:
sealion_types = ["adult_males", 
    "subadult_males",
    "adult_females",
    "juveniles",
    "pups"]

In [11]:
df_sum = df.sum()

In [12]:
counts = dict()
total = 0
for s in sealion_types:
    counts[s] = df_sum[s]
    total += 1/counts[s]

In [13]:
probas = dict()
for s, c in counts.items():
    probas[s] = (1/counts[s])/total

In [14]:
background_proba = np.min(list(probas.values()))/10

In [15]:
probas

{'adult_females': 0.048142699511667254,
 'adult_males': 0.3351506883474506,
 'juveniles': 0.089826648353188876,
 'pups': 0.11096914409391793,
 'subadult_males': 0.41591081969377525}

In [16]:
from keras.preprocessing.image import Iterator

In [17]:
import keras.backend as K

In [21]:
class PatchesIterator(Iterator):
    """Iterator yielding training samples
    :param root_dir: Directory containing training images, density map and sampling map.
    :param image_ids: Set of image ids to use to sample patches.
    :param sample_per_image: Number of patches to sample on each image.
    :param target_size: Size of the patches sampled.
    :param batch_size: Number of patches sampled per batch
    :param shuffle: Boolean, whether to shuffle the data between epochs.
    :param seed: Random seed for data shuffling.
    """

    def __init__(self, root_dir, image_ids, sample_per_image=200,
                 target_size=(224, 224),
                 batch_size=32, shuffle=True, seed=42):
                 
        self.image_ids = image_ids
        self.sample_per_image = sample_per_image
        self.target_size = target_size
        self.n_samples_per_epoch = len(image_ids) * sample_per_image
                 
        super(PatchesIterator, self).__init__(self.n_samples_per_epoch, batch_size, shuffle, seed)
        
    def __sample__(self, img, density, sampling):
        h, w, _ = img.shape
        loc_index = np.random.choice(h * w, size=(1, ), replace=False, p=sampling)
        x = loc_index % w
        y = loc_index // w
        y_start = y - self.target_size[0] // 2
        y_end = y + self.target_size[0] // 2 + 1
        x_start = x - self.target_size[1] // 2
        x_end = x + self.target_size[1] // 2 + 1
        return img[y_start:y_end, x_start:x_end, :],\
               density[y_start:y_end, x_start:x_end, :]
        
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
                 
        batch_x = np.zeros((current_batch_size, self.target_size[0], self.target_size[1], 3), dtype=K.floatx())
        batch_y = np.zeros((current_batch_size, self.target_size[0], self.target_size[1], 5), dtype=K.floatx())
        
        for i, j in enumerate(index_array):
            image_id = self.image_ids[j//self.sample_per_image]
            img = cv2.imread(os.path.join(self.root_dir, "Train", str(image_id) + ".jpg"))
            density = np.load(os.path.join(self.root_dir, "TrainDensity", str(image_id) + ".npy"))
            sampling = np.load(os.path.join(self.root_dir, "TrainSampling", str(image_id) + ".npy"))
            x, y = self.__sample__(img, density, sampling)
            batch_x[i,:,:,:] = x
            batch_y[i,:,:,:] = y  

        return batch_x, batch_y
        
        